##### Get parameters from config .yaml file

This is the standard use case for generating the roughness blocks.
The user inputs the parameters config file containing the necessary parameters.

These parameters account for the number of blocks being generated in each direction and also the offset applied to each row.

In [4]:
import pathlib
from cfdmod.use_cases.block_gen import *

cfg_file = pathlib.Path("./fixtures/tests/block_gen/block_params.yaml")
cfg = GenerationParams.from_file(cfg_file)

cfg

GenerationParams(N_blocks_x=10, N_blocks_y=10, block_params=BlockParams(height=10.0, width=10.0, length=10.0), spacing_params=SpacingParams(spacing_x=10.0, spacing_y=10.0, line_offset=10.0, is_abs=True, offset_direction=<OffsetDirection.x: 'x'>))

##### Build block and linear patterns

In order to build all the blocks, linear patterns are applied.
The first one applies to create a row in the offset direction.
The second one applies to replicate the original row and apply an offset to even rows.

In [7]:
vertices, triangles = build_single_block(cfg.block_params)

single_line_vertices, single_line_triangles = linear_pattern(
    vertices,
    triangles,
    direction=cfg.spacing_params.offset_direction,
    n_repeats=cfg.single_line_blocks,
    spacing_value=cfg.single_line_spacing,
)

full_vertices, full_triangles = linear_pattern(
    single_line_vertices,
    single_line_triangles,
    direction=cfg.perpendicular_direction,
    n_repeats=cfg.multi_line_blocks,
    spacing_value=cfg.multi_line_spacing,
    offset_value=cfg.calculate_spacing(direction=cfg.perpendicular_direction),
)

print("Single block:", f"Vertices count: {len(vertices)}", f"Triangles count: {len(triangles)}")
print("Single line", f"Vertices count: {len(single_line_vertices)}", f"Triangles count: {len(single_line_triangles)}")
print("Replicated lines", f"Vertices count: {len(full_vertices)}", f"Triangles count: {len(full_triangles)}")

Single block: Vertices count: 8 Triangles count: 10
Single line Vertices count: 80 Triangles count: 100
Replicated lines Vertices count: 800 Triangles count: 1000


##### Export generated geometry

In order to use the generated geometry, the user must export the generated vertices and triangles, using the geometry API.

In [8]:
from cfdmod.api.geometry.STL import export_stl
import pathlib

output_path = pathlib.Path("./output/block_gen")

export_stl(output_path / "blocks.stl", full_vertices, full_triangles)